In [1]:
DATABASE_PATH = '../database/abundances.db'
element_list = ['Fe', 'Ti', 'Ca', 'Si', 'Al', 'Mg', 'Na']

In [2]:
FETCH_LIMIT = None

In [3]:
import json
import sqlite3



import sqlite3
import pandas as pd
from flask import jsonify

def query_db(query, params=(), limit=None):
    db_path = DATABASE_PATH
    connection = sqlite3.connect(db_path)

    # Add LIMIT clause if a limit is specified
    if limit is not None:
        query += f" LIMIT {limit}"
        
    # Using pandas to read the query result directly into a DataFrame
    df = pd.read_sql_query(query, connection, params=params)

    connection.close()

    # Convert DataFrame to a list of dictionaries
    result_list = df.to_dict(orient='records')

    return result_list


In [4]:
query = "SELECT * FROM abundances"
data = []

In [5]:
import time

In [6]:
# Start the timer to measure the query execution time

start_time = time.time()

result = query_db(query, data,FETCH_LIMIT )


end_time = time.time()
# Print the time taken to perform the query
print(f"time to fetch : {end_time - start_time:.4f} seconds")

time to fetch : 2.8794 seconds


In [7]:
len(result) / 7

62363.0

In [8]:
import sys


# Get the size of the variable
size_in_bytes = sys.getsizeof(result)

print(f"Size of my_list: {size_in_bytes//1024**2} MB")


Size of my_list: 3 MB


In [9]:
import time
from sklearn.neighbors import KDTree
import numpy as np

points = result

In [10]:
# kernel = True

In [11]:
# Create a KDTree for each element
element_trees = {}
element_points_dict = {}

# Start the timer to measure the query execution time
start_time = time.time()

for element in element_list:
    # print("creating for element", element)
    element_points = [
        point for point in points if point['element'] == element]
    element_points_dict[element] = element_points
    element_points = [(point['lat'], point['long'])
                        for point in element_points]
    element_trees[element] = KDTree(np.array(element_points))

end_time = time.time()
# Print the time taken to perform the query
print(f"Trees creation time : {end_time - start_time:.4f} seconds")

# Query function
def get_nearby_points(query_point, radius_degrees, element):
    query_array = np.array([query_point])

    # Query the relevant tree for the specified element
    tree = element_trees.get(element)
    if tree:
        indices = tree.query_radius(query_array, r=radius_degrees)
        return [points[i] for i in indices[0]]
    else:
        return []



Trees creation time : 0.6109 seconds


In [12]:
len(element_points_dict['Mg'])

62363

## inspecting dates

In [13]:
s1 = {point['date'] for point in points}
# Convert to a list
s1 = sorted(list(s1))
s1

# Save to JSON
with open("../static/dates.json", "w") as f:
    json.dump(s1, f, indent=4)

print("Unique dates saved to 'dates.json'.")

Unique dates saved to 'dates.json'.


## histograms

In [14]:
# Initialize a dictionary to store histogram data
histograms = {}

# Define bins
bins = np.arange(0, 51, 5)  # 0-10, 10-20, ..., 90-100

# Compute histograms with normalization
for element, points in element_points_dict.items():
    # Extract abundances
    abundances = [entry["abundance"] for entry in points]
    
    # Compute histogram counts
    counts, _ = np.histogram(abundances, bins=bins)
    
    # Normalize counts
    total = sum(counts) 
    normalized_counts = (counts*100 / total).tolist() if total > 0 else counts.tolist()
    
    # Store the result in the histogram dictionary
    histograms[element] = normalized_counts

histograms


histograms

{'Fe': [72.980773856293,
  21.907220627615732,
  4.239693407950227,
  0.8723121081410452,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Ti': [50.9661177300643,
  31.06649776309671,
  15.398553629555987,
  2.568830877283004,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Ca': [0.0,
  89.25805365360871,
  9.563362891458077,
  1.1785834549332137,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Si': [30.27917194490323,
  30.74739829706717,
  23.19644661097125,
  11.450699934255889,
  3.5742347225117457,
  0.5419880377788111,
  0.1603514904671039,
  0.041691387521447014,
  0.008017574523355194,
  0.0],
 'Al': [0.0,
  9.463944967368471,
  40.48233728332505,
  50.052114234401806,
  0.001603514904671039,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Mg': [7.815531645366644,
  20.15778586661963,
  34.3055978705322,
  37.721084617481516,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'Na': [100.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]}

In [15]:
# Save to JSON
with open("../static/histograms.json", "w") as f:
    json.dump(histograms, f, indent=4)

print("Histograms computed and saved to 'histograms.json'.")

Histograms computed and saved to 'histograms.json'.


In [16]:
from sklearn.cluster import KMeans
import numpy as np



In [17]:
len(element_points)

62363

In [19]:
# raise End

## Clustering

In [ ]:
# Perform clustering to create 2k spatial clusters
coordinates = element_points
print(len(coordinates))
n_clusters = 2000
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
labels = kmeans.fit_predict(coordinates)

# Save cluster labels and centers
cluster_centers = kmeans.cluster_centers_



62363


c:\Users\pande\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


NameError: name 'all_points' is not defined

In [29]:
import numpy as np
from collections import defaultdict

# Initialize variables
M = len(element_points_dict)  # Number of elements
n_clusters = max(labels)+1  # Total distinct labels

# Create a structure to store the cluster means for each element
cluster_means_per_element = {element: [0] * n_clusters for element in element_points_dict}

# Iterate over each element and compute cluster means
for element, points in element_points_dict.items():
    # Create a list to store sums and counts for each cluster
    cluster_sums = [0] * n_clusters
    cluster_counts = [0] * n_clusters

    # Accumulate abundances for each cluster
    for point, label in zip(points, labels):
        cluster_sums[label] += point['abundance']
        cluster_counts[label] += 1

    # Compute means for each cluster
    cluster_means_per_element[element] = [
        cluster_sums[i] / cluster_counts[i] if cluster_counts[i] > 0 else 0 for i in range(n_clusters)
    ]

# Debug: Print a small sample of results
for element, means in cluster_means_per_element.items():
    print(f"Cluster means for {element}: {means[:10]}")  # Print the first 10 clusters as a sample


Cluster means for Fe: [2.949206239491854, 3.3680655728642725, 3.7651787831232015, 2.060844708048845, 5.242877282612728, 3.785736757938886, 3.204477982269005, 4.497503548711896, 9.03224018113986, 4.143954930636301]
Cluster means for Ti: [2.0237092244839263, 4.486915366858448, 8.37501008882313, 5.126188126442744, 7.335681770757095, 2.838262368313978, 1.3334064234965206, 5.724168015555772, 4.794624453456758, 6.022089542661913]
Cluster means for Ca: [6.504173086425027, 5.672883709770662, 6.64098602456197, 7.398166011796981, 8.06190093688591, 9.9050504227542, 5.7317653187654765, 7.059549634451185, 14.849275645389516, 6.68818444987908]
Cluster means for Si: [9.588276268953216, 12.07206972105789, 8.241617812350343, 10.827143597126804, 4.9631693390433265, 14.544733188811833, 9.63851097699837, 9.938724503230901, 6.247797361680647, 11.200747923173068]
Cluster means for Al: [17.853304322887706, 11.335563742157325, 11.277706760019926, 16.497918569485275, 16.852509733067485, 14.817058653459805, 14.

In [30]:
cluster_means_per_element['Fe']

[2.949206239491854,
 3.3680655728642725,
 3.7651787831232015,
 2.060844708048845,
 5.242877282612728,
 3.785736757938886,
 3.204477982269005,
 4.497503548711896,
 9.03224018113986,
 4.143954930636301,
 1.5357041261126736,
 3.2913780571847475,
 4.0225670982056325,
 7.322662784999369,
 6.690640282147244,
 3.615902989134168,
 3.51252149887898,
 7.454554948271346,
 1.7702804472315972,
 2.2447933017949446,
 3.211527237980205,
 2.8192315738227047,
 2.7868687641109355,
 3.620983962763457,
 2.514632217238058,
 5.934811756900487,
 10.989341276690377,
 3.684420941583601,
 4.307211203874706,
 1.3435812852899705,
 2.6176077473198127,
 5.519542687827871,
 8.363657146725497,
 6.392931850814319,
 3.732908465561153,
 5.992017866182114,
 2.086864534012385,
 1.920580079748232,
 3.6979371630593176,
 6.092672403466395,
 2.065667246813088,
 6.483154610965998,
 2.352944628444978,
 6.28773257199338,
 3.8007581165059108,
 2.241096652536229,
 6.666684806607637,
 4.331963398658023,
 2.5141249028012047,
 3.61946

In [31]:
cluster_centers

array([[  82.36673421,   61.01134276],
       [ -11.95112583, -113.63471667],
       [ -38.24251417,   56.72757667],
       ...,
       [   6.90155161,   63.43529032],
       [ -31.01194024,   41.70615976],
       [ -57.79262411,  -25.09440357]])

In [32]:
# import json

# # Save the elementwise_cluster_data to a JSON file
# with open('static/{element}_clusters.json', 'w') as json_file:
#     json.dump(elementwise_cluster_data, json_file, indent=4)

# print("Data saved to 'elementwise_cluster_data.json'")


In [33]:
# Initialize a dictionary to store the final result
elementwise_cluster_data = {}

# Iterate through each element and its corresponding cluster means
for element, cluster_means in cluster_means_per_element.items():
    elementwise_cluster_data[element] = [
        {
            'element': element,
            'abundance': cluster_means[i],
            'lat': cluster_centers[i][0],  # Latitude of the cluster center
            'long': cluster_centers[i][1]  # Longitude of the cluster center
        }
        for i in range(len(cluster_centers))
    ]

# Example: Print the first cluster data for a specific element (e.g., 'Fe')
print(f"First cluster for Fe: {elementwise_cluster_data['Fe'][0]}")


First cluster for Fe: {'element': 'Fe', 'abundance': 2.949206239491854, 'lat': 82.3667342105263, 'long': 61.01134276315789}


In [37]:
import json

# Save the elementwise_cluster_data to a JSON file
with open('../static/clusters.json', 'w') as json_file:
    json.dump(elementwise_cluster_data, json_file, indent=4)

print("Data saved to 'clusters.json'")


Data saved to 'clusters.json'


In [35]:
len(cluster_centers)

2000

# End of file